In [1]:
import pandas as pd
import numpy as np
# import networkx as nx
# import matplotlib.pyplot as plt
# import geopandas as gpd
import random
import pickle

In [10]:
# median migration
df = pd.read_pickle('in_migv2')
migration_col = 'migration' + str(0)
in_df = pd.DataFrame(df[0], columns=[migration_col, 'ifips', 'jfips'], dtype= int)

for year in range(1, len(df)):
    migration_col = 'migration' + str(year)
    temp_df = pd.DataFrame(df[year], columns= [migration_col, 'ifips', 'jfips'], dtype= int)
    in_df = pd.merge(in_df, temp_df,  how='left', left_on=['ifips','jfips'], right_on = ['ifips','jfips'])

in_df['median_mig'] = in_df[['migration0', 'migration1', 'migration2', 'migration3',
       'migration4', 'migration5', 'migration6', 'migration7', 'migration8',
       'migration9', 'migration10', 'migration11', 'migration12',
       'migration13', 'migration14', 'migration15', 'migration16',
       'migration17', 'migration18', 'migration19', 'migration20',
       'migration21']].median(axis=1)

median_df = in_df[['ifips', 'jfips', 'median_mig']]

Series([], Name: migration1, dtype: int64)
Series([], Name: migration2, dtype: int64)
Series([], Name: migration3, dtype: int64)
Series([], Name: migration4, dtype: int64)
Series([], Name: migration5, dtype: int64)
Series([], Name: migration6, dtype: int64)
Series([], Name: migration7, dtype: int64)
Series([], Name: migration8, dtype: int64)
Series([], Name: migration9, dtype: int64)
Series([], Name: migration10, dtype: int64)
Series([], Name: migration11, dtype: int64)
Series([], Name: migration12, dtype: int64)
Series([], Name: migration13, dtype: int64)
Series([], Name: migration14, dtype: int64)
Series([], Name: migration15, dtype: int64)
Series([], Name: migration16, dtype: int64)
Series([], Name: migration17, dtype: int64)
Series([], Name: migration18, dtype: int64)
Series([], Name: migration19, dtype: int64)
Series([], Name: migration20, dtype: int64)
Series([], Name: migration21, dtype: int64)


In [11]:
# Rename the columns to match your actual column names if necessary
source_county_col = 'ifips'
target_county_col = 'jfips'
migration_col = 'median_mig'

# Define a function to extract the state FIPS code from a county FIPS code
def county_to_state_fips(county_fips):
    return int(str(county_fips).zfill(5)[:2])

# Create a new DataFrame with the state FIPS codes for each county FIPS code
median_df['source state'] = median_df[source_county_col].apply(county_to_state_fips)
median_df['target state'] = median_df[target_county_col].apply(county_to_state_fips)

# Group by source and target state FIPS codes and sum the migration values
state_migration = median_df.groupby(['source state', 'target state'])[migration_col].sum().reset_index()
state_migration = state_migration[(state_migration['source state'] != state_migration['target state'])]

In [12]:
# Rename the columns in the new DataFrame
state_migration.columns = ['source state', 'target state', 'migration']

# Mapping to names
state_fips_to_code = {
    1: 'AL', 2: 'AK', 4: 'AZ', 5: 'AR', 6: 'CA', 8: 'CO', 9: 'CT', 10: 'DE', 11: 'DC', 12: 'FL', 13: 'GA',
    15: 'HI', 16: 'ID', 17: 'IL', 18: 'IN', 19: 'IA', 20: 'KS', 21: 'KY', 22: 'LA', 23: 'ME', 24: 'MD',
    25: 'MA', 26: 'MI', 27: 'MN', 28: 'MS', 29: 'MO', 30: 'MT', 31: 'NE', 32: 'NV', 33: 'NH', 34: 'NJ',
    35: 'NM', 36: 'NY', 37: 'NC', 38: 'ND', 39: 'OH', 40: 'OK', 41: 'OR', 42: 'PA', 44: 'RI', 45: 'SC',
    46: 'SD', 47: 'TN', 48: 'TX', 49: 'UT', 50: 'VT', 51: 'VA', 53: 'WA', 54: 'WV', 55: 'WI', 56: 'WY'
}

# Convert the state FIPS codes to state codes
state_migration['source state'] = state_migration['source state'].map(state_fips_to_code)
state_migration['target state'] = state_migration['target state'].map(state_fips_to_code)

# remove all the ones that have 0 migration
state_migration = state_migration[state_migration['migration'] != 0]

# convert migration to flux (normalize wrt median populuation (2000 + 2001)/2) https://www.iowadatacenter.org/datatables/UnitedStates/usstpopest20002002.pdf
state_code_to_population = {
    'AL': (4468912 + 4451975)/2, 'AK': (633630+627697)/2, 'AZ': (5306966+5167142)/2, 'AR': (2694698+2678668)/2, 'CA': (34600463+34010375)/2, 'CO': (4430989 + 4326758)/2, 'CT': (3434602 + 3411956) / 2, 'DE': (796599 + 786512)/2, 'DC': (573822 + 571641)/2, 'FL': (16373330 + 16051395)/2, 'GA': (8405677 + 8234373)/2,
    'HI': (1227024 + 1212670)/2, 'ID': (1320585 + 1299721)/2, 'IL': (12520227 + 12440846)/2, 'IN': (6126743 + 6091950)/2, 'IA': (2931967 + 2928742)/2, 'KS': (2702125 + 2692557)/2, 'KY': (4068816 + 4048832)/2, 'LA': (4470368 + 4469768)/2, 'ME': (1284470 + 1277284)/2, 'MD': (5386079 + 5312461) / 2, 'MA': (6401164 + 6361720)/2, 'MI': (10006266 + 9956115)/2, 'MN': (4984535 + 4934248)/2, 'MS': (2859733 + 2848829)/2, 'MO': (5637309 + 5605067)/2, 'MT': (905382 + 903416)/2, 'NE': (1720039 + 1713375)/2, 'NV': (2097722 + 2018828)/2, 'NH': (1259359 + 1240472)/2, 'NJ': (8511116 + 8433276)/2,
    'NM': (1830935 + 1821767)/2, 'NY': (19084350 + 18999760)/2, 'NC': (8206105 + 8082261)/2, 'ND': (636550 + 641131)/2, 'OH': (11389785 + 11363568)/2, 'OK': (3469577 + 3454408)/2, 'OR': (3473441 + 3431137)/2, 'PA': (12303104 + 12286107)/2, 'RI': (1059659 + 1050698)/2, 'SC': (4062125 + 4023725)/2, 'SD': (758324 + 755783)/2, 'TN': (5749398 + 5703246)/2, 'TX': (21370983 + 20955248)/2, 'UT': (2278712 + 2243406)/2, 'VT': (612978 + 609952)/2, 'VA': (7196750 + 7105900)/2, 'WA': (5993390 + 5911803)/2, 'WV': (1800975 + 1807326)/2, 'WI': (5405947 + 5374367)/2, 'WY': (493754 + 494086)/2
}

state_migration['migration_flux'] = state_migration['migration'] / state_migration['source state'].map(state_code_to_population)
state_migration['source population'] = state_migration['source state'].map(state_code_to_population)
state_migration['target population'] = state_migration['target state'].map(state_code_to_population)

cols = ['source state', 'target state']

# Add the state distance data to state_migration
distances_df = pd.read_csv('./state_distances.csv')
distances_df.columns = ['source state', 'target state', 'distance']
state_migration = pd.merge(state_migration, distances_df,  how='left', left_on=cols, right_on=cols)

# Add the state disaster and program data to state_migration
disasters_df = pd.read_pickle('./disasters_mean_median.pkl')

# Merge disasters_df and state_migration on state so that it creates an extra column for disasters of each state
state_migration = pd.merge(state_migration, disasters_df,  how='left', left_on='source state', right_on='state', suffixes=('_source', '_target'))
state_migration = pd.merge(state_migration, disasters_df,  how='left', left_on='target state', right_on='state', suffixes=('_source', '_target'))

Create an empty directed graph
G_state = nx.DiGraph()

# Iterate through the rows in the DataFrame and add edges to the graph
for index, row in state_migration.iterrows():
    source = row['source state']
    target = row['target state']
    weight = row['migration_flux']
    G_state.add_edge(source, target, weight=weight)

# Print the graph's nodes and edges
# print("Nodes:", G_state.nodes())
# print("Edges:", G_state.edges(data=True))
with open("./median_graph.p", 'wb') as f:
    pickle.dump(G_state, f)

with open("./state_mig_median.pkl", 'wb') as f:
    pickle.dump(state_migration, f)

KeyError: "['state_source', 'state_target'] not found in axis"

In [25]:
# Hurricane Katrina happened in August 2005, so we should look at 2006 data

# 2006 migration
df = pd.read_pickle('in_migv2')
migration_col = 'migration_' + str(2006)
df_2006 = pd.DataFrame(df[16], columns=[migration_col, 'ifips', 'jfips'], dtype= int)

# Rename the columns to match your actual column names if necessary
source_county_col = 'ifips'
target_county_col = 'jfips'
migration_col = 'migration_2006'

# Define a function to extract the state FIPS code from a county FIPS code
def county_to_state_fips(county_fips):
    return int(str(county_fips).zfill(5)[:2])

# Create a new DataFrame with the state FIPS codes for each county FIPS code
df_2006['source state'] = df_2006[source_county_col].apply(county_to_state_fips)
df_2006['target state'] = df_2006[target_county_col].apply(county_to_state_fips)

# Group by source and target state FIPS codes and sum the migration values
state_migration_2006 = df_2006.groupby(['source state', 'target state'])[migration_col].sum().reset_index()
state_migration_2006 = state_migration_2006[(state_migration_2006['source state'] != state_migration_2006['target state'])]

In [26]:
# Rename the columns in the new DataFrame
state_migration_2006.columns = ['source state', 'target state', 'migration']

# Mapping to names
state_fips_to_code = {
    1: 'AL', 2: 'AK', 4: 'AZ', 5: 'AR', 6: 'CA', 8: 'CO', 9: 'CT', 10: 'DE', 11: 'DC', 12: 'FL', 13: 'GA',
    15: 'HI', 16: 'ID', 17: 'IL', 18: 'IN', 19: 'IA', 20: 'KS', 21: 'KY', 22: 'LA', 23: 'ME', 24: 'MD',
    25: 'MA', 26: 'MI', 27: 'MN', 28: 'MS', 29: 'MO', 30: 'MT', 31: 'NE', 32: 'NV', 33: 'NH', 34: 'NJ',
    35: 'NM', 36: 'NY', 37: 'NC', 38: 'ND', 39: 'OH', 40: 'OK', 41: 'OR', 42: 'PA', 44: 'RI', 45: 'SC',
    46: 'SD', 47: 'TN', 48: 'TX', 49: 'UT', 50: 'VT', 51: 'VA', 53: 'WA', 54: 'WV', 55: 'WI', 56: 'WY'
}

# Convert the state FIPS codes to state codes
state_migration_2006['source state'] = state_migration_2006['source state'].map(state_fips_to_code)
state_migration_2006['target state'] = state_migration_2006['target state'].map(state_fips_to_code)

# remove all the ones that have 0 migration
state_migration_2006 = state_migration_2006[state_migration_2006['migration'] != 0]

# convert migration to flux (normalize wrt 2006 population) https://fred.stlouisfed.org/release/tables?rid=118&eid=259194&od=2006-01-01
state_code_to_population = {
    'AL': 4628981, 'AK': 675302, 'AZ': 6029141, 'AR': 2821761, 'CA': 36021202, 'CO': 4720423, 'CT': 3517460, 'DE': 859268, 'DC': 570681, 'FL': 18166990, 'GA': 9155813,
    'HI': 1309731, 'ID': 1468669, 'IL': 12643955, 'IN': 6332669, 'IA': 2982644, 'KS': 2762931, 'KY': 4219239, 'LA': 4302665, 'ME': 1323619, 'MD': 5627367, 'MA': 6410084, 'MI': 10036081, 'MN': 5163555, 'MS': 2904878, 'MO': 5842704, 'MT': 952692, 'NE': 1772693, 'NV': 2522658, 'NH': 1308389, 'NJ': 8661679, 'NM': 1962137, 'NY': 19104631, 'NC': 8917270, 'ND': 649422, 'OH': 11481213, 'OK': 3594090, 'OR': 3670883, 'PA': 12510809, 'RI': 1063096, 'SC': 4357847, 'SD': 783033, 'TN': 6088766, 'TX': 23359580, 'UT': 2525507, 'VT': 622892, 'VA': 7673725, 'WA': 6370753, 'WV': 1827912, 'WI': 5577655, 'WY': 522667
}

state_migration_2006['migration_flux'] = state_migration_2006['migration'] / state_migration_2006['source state'].map(state_code_to_population)
state_migration_2006['source population'] = state_migration_2006['source state'].map(state_code_to_population)
state_migration_2006['target population'] = state_migration_2006['target state'].map(state_code_to_population)

# Add the state distance data to state_migration
distances_df = pd.read_csv('./state_distances.csv')
distances_df.columns = ['source state', 'target state', 'distance']
state_migration_2006 = pd.merge(state_migration_2006, distances_df,  how='left', left_on=cols, right_on=cols)

# Add the state disaster and program data to state_migration
disasters_df = pd.read_pickle('./disasters_2006.pkl')

# Merge disasters_df and state_migration on state so that it creates an extra column for disasters of each state
state_migration_2006 = pd.merge(state_migration_2006, disasters_df,  how='left', left_on='source state', right_on='state', suffixes=('_source', '_target')).fillna(0)
state_migration_2006 = pd.merge(state_migration_2006, disasters_df,  how='left', left_on='target state', right_on='state', suffixes=('_source', '_target')).fillna(0)

# Create an empty directed graph
G_state = nx.DiGraph()

# Iterate through the rows in the DataFrame and add edges to the graph
for index, row in state_migration_2006.iterrows():
    source = row['source state']
    target = row['target state']
    weight = row['migration_flux']
    G_state.add_edge(source, target, weight=weight)

# Print the graph's nodes and edges
print("Nodes:", G_state.nodes())
print("Edges:", G_state.edges(data=True))
with open("/Users/zasghar.19/Desktop/eeps1720/NetworkAnalysis/graph_2006.p", 'wb') as f:
    pickle.dump(G_state, f)

with open("./state_mig_2006.pkl", 'wb') as f:
    pickle.dump(state_migration_2006, f)